In [126]:
import re
import pandas as pd

from energy_ranges import (
    ERange,
    KRange,
    merge_ranges,
    wavenumber_to_energy,
)

# Dev

In [141]:
def clean_column_names(columns):
    # Convert to lowercase, replace spaces with underscores, and remove non-alphanumeric characters
    return [re.sub(r'[^\w]', '', col.lower().replace(' ', '_')) if isinstance(col, str) else col for col in columns]


def convert_energy_range(energy_range_str):
    """Converts the energy range string into a list of numeric values, applying convert_k_to_E where needed."""
    energy_list = []
    for item in energy_range_str.split():
        if 'k' in item:
            # Remove the 'k' and apply the convert_k_to_E function
            k = float(item.replace('k', ''))
            E = wavenumber_to_energy(k)
            energy_list.append(E)
        else:
            # Convert to float for other values
            energy_list.append(float(item))
    return energy_list

def convert_energy_steps(energy_steps_str):
    """Converts the energy steps string into a list, applying E_step_to_k_step where needed."""
    energy_steps_list = []
    for item in energy_steps_str.split():
        if 'k' in item:
            value = float(item.replace('k', ''))
            energy_steps_list.append(value)
        else:
            # Convert to float for other values
            energy_steps_list.append(float(item))
    return energy_steps_list

def check_wheel_type(file_name, printit=True):
    # Extract the wheel type from the first row, first column
    wheel_type = pd.read_excel(file_name, header=None).iloc[0, 0]
    if printit:
        print("wheel_type: ", wheel_type)
    return wheel_type

def validate_data(df):
    validated_df = df.copy()

    # Convert 'Yes' to True and 'No' to False in specific columns
    yes_no_columns = ['measure_this_slot', 'is_standard']
    for col in yes_no_columns:
        if col in validated_df.columns:
            validated_df[col] = validated_df[col].map({'Yes': True, 'No': False})

    # Ensure these columns are string types
    string_columns = ['file_name', 'plotting_mode', 'element_symbol', 'sample_name', 'absorption_edge']
    validated_df[string_columns] = validated_df[string_columns].astype(str)

    # Ensure these columns are integer types, coerce invalid entries to NaN
    int_columns = ['note', 'kweight', 'detector_x', 'sample_x', 'sample_y', 
                   'slit_width', 'slit_height']
    validated_df[int_columns] = validated_df[int_columns].apply(pd.to_numeric, errors='coerce', downcast='integer')

    # Ensure these columns are float types, coerce invalid entries to NaN
    float_columns = ['integration_times', 'detector_x', 'sample_x', 'sample_y', 
                     'slit_width', 'slit_height']
    validated_df[float_columns] = validated_df[float_columns].apply(pd.to_numeric, errors='coerce')

    # Convert energy_range strings into energy list using the previously defined convert_energy_range function
    if 'energy_range' in validated_df.columns:
        validated_df['energy_range'] = validated_df['energy_range'].apply(convert_energy_range)

    # Convert energy_steps strings into energy step list using the previously defined convert_energy_steps function
    if 'energy_steps' in validated_df.columns:
        validated_df['energy_steps'] = validated_df['energy_steps'].apply(convert_energy_steps)

    # Validate that plotting_mode is either 'transmission' or 'fluorescence'
    if 'plotting_mode' in validated_df.columns:
        valid_modes = {'transmission', 'fluorescence'}
        if not validated_df['plotting_mode'].isin(valid_modes).all():
            raise ValueError("Invalid plotting_mode. It must be either 'transmission' or 'fluorescence'.")

    # Validate that absorption_edge is either 'K', 'L1', 'L2', or 'L3'
    if 'absorption_edge' in validated_df.columns:
        valid_edges = {'K', 'L1', 'L2', 'L3'}
        if not validated_df['absorption_edge'].isin(valid_edges).all():
            raise ValueError("Invalid absorption_edge. It must be 'K', 'L1', 'L2', or 'L3'.")

    return validated_df


## pytest to test above functions

In [132]:
file_name = 'standard_sample_wheel.xlsx'

# Read the Excel file with header at row 7 and skip row 8 (example lines)
df = pd.read_excel(file_name, header=7, skiprows=[8])

# Clean the column names by replacing symbols with underscores
df.columns = clean_column_names(df.columns)

# Check the wheel type
wheel_type = check_wheel_type(file_name)
if wheel_type.lower() == 'standard':
    num_slots = 12

# Select the first `num_slots` rows of data
data = df.head(num_slots)

# validate the data
data = validate_data(data)

# Convert the DataFrame to a list of dictionaries
list_of_dicts = data.to_dict(orient='records')

# Add 'wheel' and 'slot_num' information to each dictionary
for idx, item in enumerate(list_of_dicts):
    item['wheel'] = wheel_type
    item['slot_num'] = idx + 1  # Assuming slot_num starts from 1


wheel_type:  Standard


In [140]:
print(data['energy_steps'][11])

[21.0, 2.0, 0.3, 0.05]
